In [1]:
# Беглый анализ колонки и удаление, если много пропусков
col_for_drop = []

def col_info_train(col):

    if col.isnull().sum() > 0:
        percent_isnull = (col.isnull().sum() / len(col)) * 100
        print(col.name)
        print('Количество пропусков: {}'.format(col.isnull().sum()))
        print('% пропусков: {}'.format(percent_isnull))
        print('---')       
        
    
        # Удалю признак, в котором есть пропусков больше 10%
        if percent_isnull > 10:
            col_for_drop.append(col.name)
            train.drop([col.name], axis='columns', inplace=True)
        

for col in train:
    col_info_train(train[col]) 

In [ ]:
# Просмотр гистограммы для признака
def feature_hist(data, col):
    sns.countplot(data[ col])
    plt.title('Histogram for default')

In [ ]:
# Беглый просмотр данных в признаке
def col_info(data, col, bins):
    percent_isnull = (data[col].isnull().sum()) / len(data[col]) * 100
    print('Название колонки: {}'.format(data[col].name))
    print('Количество пропусков: {}'.format(data[col].isnull().sum()))
    print('% пропусков: {}'.format(round(percent_isnull, 2)))
    print('{},'.format(data[col].describe()))
    print('Распределение:\n{},'.format(data[col].value_counts()))
    data[col].hist(bins=bins)

In [ ]:
# Визуализация признаков и пропусков
def viz_na(data):
    """NA visualisation"""
    global cols
    cols = data.columns # запишем названия строки сделаем переменную глобальной
    # определяем цвета 
    # желтый - пропущенные данные, синий - не пропущенные
    colours = ['#000099', '#ffff00'] 
    sns.heatmap(data[cols].isnull(), cmap=sns.color_palette(colours))
    plt.show()


viz_na(train)
viz_na(test)

In [ ]:
# Статистика пропусков 
def stat_na_per_percent(data):
    print(f'{data.shape}')
    for col in data.columns:
        pct_missing = np.mean(data[col].isnull())
        if pct_missing > 0: # выведем только те, где больше 0
            print('{} - {}%'.format(col, round(pct_missing*100)))
    print("END", end = '\n\n')
    
stat_na_per_percent(train)
stat_na_per_percent(test)

In [ ]:
# Пропущенные данные и выбросы

# В добавок к некорректному определению типов данных, другая частая проблема — это пропуски в данных.
# У наличия пропусков могут быть разные причины, но пропуски нужно либо заполнить, либо исключить из набора полностью.

# Для начала попробуем оценить масштаб проблемы

# Функция считает ошибочные данные в колонке
def missing_values_table(df):
        # Всего ошибочных данных (это я уже сделал выше, но выведу их и здесь для наглядности)
        mis_val = df.isnull().sum()
        
        # Процент ошибочных данных
        mis_val_percent = 100 * df.isnull().sum() / len(df)
        
        # Тип данных каждой колонки
        sm_cols = [c for c in sm.columns]

        cat_dtypes_dict = {}

        for ct in sm_cols:
            cat_dtypes_dict.update({sm[ct].name: sm[ct].dtype})

        cat_dtypes = pd.Series(cat_dtypes_dict) 
      
        # Сделаем датафрейм с результатами
        mis_val_table = pd.concat([mis_val, mis_val_percent, cat_dtypes], axis=1)
        
        # Переназовем колонки
        mis_val_table_ren_columns = mis_val_table.rename(
        columns = {0 : 'Missing Values', 1 : '% of Total Values', 2 : 'Types'})
        
#         # Отсортируем данные и исключим колонки, в которых нет пропусков
#         mis_val_table_ren_columns = mis_val_table_ren_columns[
#             mis_val_table_ren_columns.iloc[:,1] != 0].sort_values(
#         '% of Total Values', ascending=False).round(1)

        # Отсортируем данные с учетом колонок, в которых нет пропусков
        mis_val_table_ren_columns = mis_val_table_ren_columns.sort_values(
        '% of Total Values', ascending=False).round(1)
        
        # Выведем итоговую информацию
        print ("Your selected dataframe has " + str(df.shape[1]) + " columns.\n"      
            "There are " + str(mis_val_table_ren_columns.shape[0]) +
              " columns that have missing values.")
        
        # Вернем датафрейм с информацией об ошибочных данных
        return mis_val_table_ren_columns
    
missing_values_table(sm)

In [ ]:
# Выделим числовые и категориальные признаки в отдельные листы:

categorical_columns = [c for c in sm.columns if sm[c].dtype.name == 'object']
numerical_columns   = [c for c in sm.columns if sm[c].dtype.name != 'object']
print('Категориальные признаки:', categorical_columns)
print('Числовые признаки:', numerical_columns)
                       